In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb)
# **Connect to Cloud SQL using the Cloud SQL Python Connector**

---

This notebook will be demonstrating how to connect and query data from a Cloud SQL database in an easy and efficient way all from within a jupyter style notebook! Let's have some fun!


### 📒 Using this interactive notebook

Click the **run** icons ▶️  of each section within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

## 🐍  **Cloud SQL Python Connector**
To connect and access our Cloud SQL database instance(s) we will leverage the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector).

The Cloud SQL Python Connector is a library that can be used alongside a database driver to allow users to easily connect to a Cloud SQL database without having to manually allowlist IP or manage SSL certificates. 🥳 🎉 🤩

### ♥️ Benefits of Using a Connector
Using a Cloud SQL connector provides the following benefits:

- 🔑 **IAM Authorization**: uses IAM permissions to control who/what can connect to your Cloud SQL instances.
- 🔒 **Improved Security**: uses robust, updated TLS 1.3 encryption and identity verification between the client connector and the server-side proxy, independent of the database protocol.
- 👍 **Convenience**: removes the requirement to use and distribute SSL certificates, as well as manage firewalls or source/destination IP addresses.
- 🪪 **IAM DB Authentication** (optional): provides support for Cloud SQL’s automatic IAM DB AuthN feature.

### 📱 Supported Dialects/Drivers
Google Cloud SQL and the Python Connector currently support the following dialects of SQL: **MySQL**, **PostgreSQL**, and **SQL Server**.

Depending on which dialect you are using for your relational database(s) the Python Connector will utilize a different database driver.

SUPPORTED DRIVERS:

*   **pymysql** (MySQL) 🐬
*   **pg8000** (PostgreSQL) 🐘
*   **pytds** (SQL Server) 🗄

Therefore, depending on the dialect of your database you will need to switch to the corresponding notebook!

📗 [**MySQL Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb) (this notebook)

📘 [**PostgreSQL Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/postgres_python_connector.ipynb)

📕 [**SQL Server Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/sqlserver_python_connector.ipynb)

## 🚧 **Getting Started**
This notebook requires the following steps to be completed in order to successfully make Cloud SQL connections with the Cloud SQL Python Connector.

### 🔐 Authenticate to Google Cloud within Colab
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [13]:
from google.colab import auth

auth.authenticate_user()

### 🔗 Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab. 🏅 😀

In [16]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "dream-to-stream" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


### ☁ Configure Your Google Cloud Project
Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the
[Cloud SQL Client][client-role] role.

> 🚨 The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [17]:
# grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [dream-to-stream].
bindings:
- members:
  - user:adzlam116@gmail.com
  - user:mohammad.a.zayyad@gmail.com
  - user:rohansreenivasan5@gmail.com
  role: roles/cloudsql.client
- members:
  - serviceAccount:service-10869121041@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:10869121041-compute@developer.gserviceaccount.com
  - serviceAccount:10869121041@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - user:Divya.manirajan@gmail.com
  - user:adzlam116@gmail.com
  - user:mohammad.a.zayyad@gmail.com
  - user:rohansreenivasan5@gmail.com
  role: roles/owner
etag: BwX1ozkRfXs=
version: 1


2. Enable the [Cloud SQL Admin API][admin-api] within your project.

[admin-api]: https://console.cloud.google.com/apis/api/sqladmin.googleapis.com

In [18]:
# enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

## ☁️ Setting up Cloud SQL
A **MySQL** Cloud SQL instance is required for the following stages of this notebook.

###💽 **Create a MySQL Instance**
Running the below cell will verify the existence of a Cloud SQL instance or create a new one if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [19]:
#@markdown Please fill in the both the Google Cloud region and name for your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_name = "dream-to-stream-instance" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for database 'root' user: ")
  !gcloud sql instances create {instance_name} --database-version=MYSQL_8_0 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql_iam_authentication=On

Found existing MySQL Cloud SQL Instance!


### 🥪 Create a Sandwiches Database
A `sandwiches` database will be used in later steps when connecting to and querying a Cloud SQL database.

To create a `sandwiches` database within your Cloud SQL instance run the below command:

In [ ]:
!gcloud sql databases create sandwiches --instance={instance_name}

ERROR: (gcloud.sql.databases.create) HTTPError 400: Invalid request: couldn't create database: Error 1007: Can't create database 'sandwiches'; database exists.


### 🧑‍🍳 Create a Chef Database User
To create the `chef` database user that is used throughout the notebook, run the following `gcloud` command.

In [ ]:
!gcloud sql users create chef \
  --instance={instance_name} \
  --password="food"

<img src='https://i.pinimg.com/originals/12/64/dd/1264dd5ff31fbc65c5edbb5e1a71830e.gif' class="center"/>


## 🐍 Python Connector Usage
Let's now connect to Cloud SQL using the Python Connector! 🚀 ⭐ 🐍

### 🎟 **Configuring Credentials**
The Cloud SQL Python Connector uses [**Application Default Credentials (ADC)**](https://cloud.google.com/docs/authentication) strategy for resolving credentials.

> 💡 Using the Python Connector in Cloud Run, App Engine, or Cloud Functions will automatically use the service account deployed with each service, allowing this step to be skipped. ✅

Please see the [google.auth](https://google-auth.readthedocs.io/en/master/reference/google.auth.html) package documentation for more information on how these credentials are sourced.

This means setting default credentials was previously done for you when you ran:
```python
from google.colab import auth

auth.authenticate_user()
```


### 💻 **Install Code Dependencies**
It is recommended to use the Connector alongside a library that can create connection pools, such as [SQLAlchemy](https://www.sqlalchemy.org/).
This will allow for connections to remain open and be reused, reducing connection overhead and the number of connections needed

Let's `pip install` the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector) as well as [SQLAlchemy](https://www.sqlalchemy.org/), using the below command.

In [20]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 🐬 **Connect to a MySQL Instance**
We are now ready to connect to a MySQL instance using the Cloud SQL Python Connector! 🐍 ⭐ ☁


Let's set some parameters that are needed to connect properly to a Cloud SQL instance:
*   `INSTANCE_CONNECTION_NAME` : The connection name to your Cloud SQL Instance, takes the form `PROJECT_ID:REGION:INSTANCE_NAME`.
*   `DB_USER` : The user in which the connector will use to connect to the database.
*   `DB_PASS` : The password of the DB_USER.
*   `DB_NAME` : The name of the database on the Cloud SQL instance to connect to.

In [21]:
# initialize parameters
INSTANCE_CONNECTION_NAME = f"dream-to-stream:us-central1:dream-to-stream-instance" # i.e demo-project:us-central1:demo-instance
#print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "root"
DB_PASS = "dreamToStream"
DB_NAME = "dreamToStream"

### ✅ **Basic Usage**
To connect to Cloud SQL using the connector, initialize a `Connector` object and call its `connect` method with the proper input parameters.

The `connect` method takes in the parameters we previously defined, as well as a few additional parameters such as:
*   `driver`: The name of the database driver to connect with.
*   `ip_type` (optional): The IP type (public or private)  used to connect. IP types can be either `IPTypes.PUBLIC` or `IPTypes.PRIVATE`. ([Example](#scrollTo=yjAPpIDdRfu2))
*   `enable_iam_auth`: (optional) Boolean enabling IAM based authentication. ([Example](#scrollTo=GpVKrv0TCXje))

Let's show an example! 🤘 🙌

In [22]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

In [ ]:
# This is what I used to load data for Platform
# connect to connection pool
# PLATFORM INSERT:
with pool.connect() as db_conn:
    # create ratings table in our sandwiches database
    import csv
    import re
    
    filename = '/platform.csv'

    with open(filename, 'r') as csvfile:
      datareader = csv.reader(csvfile)
      insert_stmt_platform = sqlalchemy.text("INSERT INTO Platform (platformId, country, platform) VALUES (:platformId, :country, :platform)")

      flag = 1
      for row in datareader:
          #for platformId, platform, country
          #    0        1        2        
          if flag == 1:
            flag = 0
            continue

          else:
            db_conn.execute(insert_stmt_platform, parameters={"platformId": str(row[0]), "country": str(row[2]), "platform": str(row[1])})
            # db_conn.commit()

    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM Platform")).fetchall()

    # show results
    for row in results:
      print(row)

In [ ]:
# SHOW INSERT
with pool.connect() as db_conn:
    # create ratings table in our sandwiches database
    import csv

    filename = '/shows.csv'

    with open(filename, 'r') as csvfile:
      datareader = csv.reader(csvfile)
      # Skip the header row
      next(datareader)

      # Prepare the insert statement
      insert_stmt = sqlalchemy.text("INSERT INTO Shows (name, country, releaseYear, tvRating, seasons, genre, platform) VALUES (:name, :country, :releaseYear, :tvRating, :seasons, :genre, :platform)")

      # Prepare the data for bulk insertion
      data = [{'name': row[0], 'country': row[3], 'releaseYear': row[4], 'tvRating': row[5], 'seasons': int(row[6]), 'genre': row[7], 'platform': row[9]} for index, row in enumerate(datareader)]

      # Execute the bulk insertion
      db_conn.execute(insert_stmt, data)
      # db_conn.commit()

    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM Shows")).fetchall()

    # show results
    for row in results:
      print(row)


In [ ]:
# MOVIES INSERT
with pool.connect() as db_conn:
    # create ratings table in our sandwiches database
    import csv
    import re

    def extract_numbers(string):
        pattern = r'\d+' # \d matches any digit, + matches one or more occurrences
        numbers = re.findall(pattern, string) # find all non-overlapping matches
        return ''.join(numbers) # combine the matched numbers into a string

    filename = '/movies.csv'

    with open(filename, 'r') as csvfile:
      datareader = csv.reader(csvfile)
      # Skip the header row
      next(datareader)

      # Prepare the insert statement
      insert_stmt = sqlalchemy.text("INSERT INTO Movies (name, country, releaseYear, tvRating, length, genre, platform) VALUES (:name, :country, :releaseYear, :tvRating, :length, :genre, :platform)")

      # Prepare the data for bulk insertion
      data = [{'name': row[0], 'country': row[3], 'releaseYear': row[4], 'tvRating': row[5], 'length': int(row[6]), 'genre': row[7], 'platform': row[9]} for index, row in enumerate(datareader)]

      # Execute the bulk insertion
      db_conn.execute(insert_stmt, data)
    #   db_conn.commit()
    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM Movies")).fetchall()

    # show results
    for row in results:
      print(row)

In [ ]:
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  import csv

  filename = 'all_movies.csv'

  with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    insert_stmt = sqlalchemy.text(
       "INSERT INTO Movies (name, length, genre, rating, releaseYear, location, serverId) VALUES (:name, :length, :genre, :rating, :releaseYear, :location, :serverId)",
   )
  
  import re

def extract_numbers(string):
    pattern = r'\d+' # \d matches any digit, + matches one or more occurrences
    numbers = re.findall(pattern, string) # find all non-overlapping matches
    return ''.join(numbers) # combine the matched numbers into a string

  # PLATFORM INSERT:
  #   insert_stmt_platform = sqlalchemy.text(
  #      "INSERT INTO Platform (serverId, availability, platform) VALUES (:serverId, :availability, :platform)",
  #  )
  #   for i in range(17000):
  #       db_conn.execute(insert_stmt_platform, parameters={"serverId": i, "availability": "available", "platform": "netflix"})
  #       db_conn.commit()
    
    flag = 1
    for row in datareader:
        #for name, location, releaseYear, rating, length, genre, service, serverId in row:
        #    0        1        2            3       4      5      6         7
        if flag == 1:
          flag = 0
          continue

        else:
          
          db_conn.execute(insert_stmt, parameters={"name": str(row[0]), "length": int(extract_numbers(row[4])), "genre": row[5], "rating": (row[3]), "releaseYear": (row[2]), "location": str(row[1]), "serverId": (row[7])})
          db_conn.commit()
  # db_conn.execute(
  #   sqlalchemy.text(
  #     "CREATE TABLE IF NOT EXISTS ratings "
  #     "( id SERIAL NOT NULL, name VARCHAR(255) NOT NULL, "
  #     "origin VARCHAR(255) NOT NULL, rating FLOAT NOT NULL, "
  #     "PRIMARY KEY (id));"
  #   )
  # )
  # # insert data into our ratings table
  # insert_stmt = sqlalchemy.text(
  #     "INSERT INTO ratings (name, origin, rating) VALUES (:name, :origin, :rating)",
  # )

  # # insert entries into table
  # db_conn.execute(insert_stmt, parameters={"name": "HOTDOG", "origin": "Germany", "rating": 7.5})
  # db_conn.execute(insert_stmt, parameters={"name": "BÀNH MÌ", "origin": "Vietnam", "rating": 9.1})
  # db_conn.execute(insert_stmt, parameters={"name": "CROQUE MADAME", "origin": "France", "rating": 8.3})
  # db_conn.commit()

  # # query and fetch ratings table
  # results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

  # # show results
  # for row in results:
  #   print(row)


To use this connector with SQLAlchemy, we use the `creator` argument for `sqlalchemy.create_engine`

Now that we have established a connection pool, let's write a query! 🎉 📝

In [ ]:
#Movie Insert
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  import csv
  import re

  def extract_numbers(string):
      pattern = r'\d+' # \d matches any digit, + matches one or more occurrences
      numbers = re.findall(pattern, string) # find all non-overlapping matches
      return ''.join(numbers) # combine the matched numbers into a string

  filename = 'all_movies.csv'

  with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    insert_stmt = sqlalchemy.text(
       "INSERT INTO Movies (name, length, genre, rating, releaseYear, location, serverId) VALUES (:name, :length, :genre, :rating, :releaseYear, :location, :serverId)",
   )
      
    flag = 1
    for row in datareader:
        #for name, location, releaseYear, rating, length, genre, service, serverId in row:
        #    0        1        2            3       4      5      6         7
        if flag == 1:
          flag = 0
          continue

        else:
          
          db_conn.execute(insert_stmt, parameters={"name": str(row[0]), "length": int(extract_numbers(row[4])), "genre": row[5], "rating": (row[3]), "releaseYear": (row[2]), "location": str(row[1]), "serverId": (row[7])})
          db_conn.commit()


In [ ]:
#TV INSERT
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  import csv
  import re

  def extract_numbers(string):
      pattern = r'\d+' # \d matches any digit, + matches one or more occurrences
      numbers = re.findall(pattern, string) # find all non-overlapping matches
      return ''.join(numbers) # combine the matched numbers into a string

  filename = 'all_tv.csv'

  with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    insert_stmt = sqlalchemy.text(
       "INSERT INTO Shows (name, length, genre, rating, releaseYear, location, serverId) VALUES (:name, :length, :genre, :rating, :releaseYear, :location, :serverId)",
   )
      
    flag = 1
    for row in datareader:
        #for name, location, releaseYear, rating, length, genre, service, serverId in row:
        #    0        1        2            3       4      5      6         7
        if flag == 1:
          flag = 0
          continue

        else:
          
          db_conn.execute(insert_stmt, parameters={"name": str(row[0]), "length": int(extract_numbers(row[4])), "genre": row[5], "rating": (row[3]), "releaseYear": (row[2]), "location": str(row[1]), "serverId": (row[7])})
          db_conn.commit()


In [ ]:
#Director INSERT
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  import csv
  import re

  def extract_numbers(string):
      pattern = r'\d+' # \d matches any digit, + matches one or more occurrences
      numbers = re.findall(pattern, string) # find all non-overlapping matches
      return ''.join(numbers) # combine the matched numbers into a string

  filename = 'cast_movie.csv'

  with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    insert_stmt = sqlalchemy.text(
       "INSERT IGNORE INTO Cast (firstName, lastName) VALUES (:firstName, :lastName)",
   )
      
    flag = 1
    for row in datareader:
        #for name, location, releaseYear, rating, length, genre, service, serverId in row:
        #    0        1        2            3       4      5      6         7
        if flag == 1:
          flag = 0
          continue

        else:
          
          db_conn.execute(insert_stmt, parameters={"firstName": str(row[0]), "lastName": str((row[1]))})
          db_conn.commit()


In [ ]:
# MovieRating Insert
with pool.connect() as db_conn:
  from pandas import *
  import random
  import sqlalchemy

  filename = '/data_migration/shows.csv'

  with open(filename, 'r') as csvfile:
      datareader = pandas.read_csv(csvfile)
      # Skip the header row
      title = datareader['title'].tolist()

      insert_stmt = sqlalchemy.text("INSERT INTO ShowRating (id, name, date, value, comments) VALUES (:id, :name, :date, :value, :comments)")

      with pool.connect() as db_conn:
         # Query the Movies table to get all valid name values
          movies_query = sqlalchemy.text("SELECT name FROM Movies")
          valid_names = set(row[0] for row in db_conn.execute(movies_query).fetchall())

          # Prepare the data for bulk insertion
          data = [{'id': random.randint(1, 14), 'name': title[random.randint(0, len(title)-1)], 'date': '2023-03-06', 'value': random.randint(0, 10), 'comments': 'Good watch'} for _ in range(5000)]
          new_data = []
          for elem in data:
              if not any(d['id'] == elem['id'] and d['name'] == elem['name'] for d in new_data):
                  new_data.append(elem)
              else:
                  print(f"Skipping duplicate row: {elem}")

          data = new_data

          # Filter out any rows with invalid name values
          valid_data = [row for row in data if row['name'] in valid_names]

          # Insert the unique data
          db_conn.execute(insert_stmt, valid_data)
          #db_conn.commit()

          # query and fetch ratings table
          results = db_conn.execute(sqlalchemy.text("SELECT * FROM ShowRating")).fetchall()

          # show results
          for row in results:
              print(row)

In [ ]:
# ShowRating Insert
with pool.connect() as db_conn:
  from pandas import *
  import random
  import sqlalchemy

  filename = '/data_migration/shows.csv'

  with open(filename, 'r') as csvfile:
      datareader = pandas.read_csv(csvfile)
      # Skip the header row
      title = datareader['title'].tolist()

      insert_stmt = sqlalchemy.text("INSERT INTO ShowRating (id, name, date, value, comments) VALUES (:id, :name, :date, :value, :comments)")

      with pool.connect() as db_conn:
         # Query the Movies table to get all valid name values
          show_query = sqlalchemy.text("SELECT name FROM Shows")
          valid_names = set(row[0] for row in db_conn.execute(show_query).fetchall())

          # Prepare the data for bulk insertion
          data = [{'id': random.randint(1, 14), 'name': title[random.randint(0, len(title)-1)], 'date': '2023-03-06', 'value': random.randint(0, 10), 'comments': 'Good watch'} for _ in range(5000)]
          new_data = []
          for elem in data:
              if not any(d['id'] == elem['id'] and d['name'] == elem['name'] for d in new_data):
                  new_data.append(elem)
              else:
                  print(f"Skipping duplicate row: {elem}")

          data = new_data

          # Filter out any rows with invalid name values
          valid_data = [row for row in data if row['name'] in valid_names]

          # Insert the unique data
          db_conn.execute(insert_stmt, valid_data)
          #db_conn.commit()

          # query and fetch ratings table
          results = db_conn.execute(sqlalchemy.text("SELECT * FROM ShowRating")).fetchall()

          # show results
          for row in results:
              print(row)

You have successfully been able to connect to a Cloud SQL instance from this notebook and make a query. YOU DID IT! 🕺 🎊 💃

<img src=https://media.giphy.com/media/MtHGs1yo4FFKrIs55L/giphy.gif />

To close the `Connector` object's background resources, call it's `close() ` method at the end of your code as follows:


In [ ]:
# cleanup connector object
connector.close()

### 🪪  IAM Database Authentication

[Automatic IAM database authentication](https://cloud.google.com/sql/docs/mysql/authentication#automatic) is supported for **MySQL** Cloud SQL instances.

> 💡 This allows an IAM user to establish an authenticated connection to a MySQL database without having to set a password and enabling the `enable_iam_auth` parameter in the connector's `connect` method.

> 🚨 If you are using a pre-existing Cloud SQL instance within this notebook you may need to [configure Cloud SQL instance to allow IAM authentication](https://cloud.google.com/sql/docs/mysql/create-edit-iam-instances#configuring_existing_instances_for) by setting the `cloudsql_iam_authentication` database flag to `On`.
(Cloud SQL instances created within this notebook already have it enabled)


IAM principals wanting to use IAM authentication to connect to a Cloud SQL instance require the `Cloud SQL Instance User` and `Cloud SQL Client` IAM role.

Let's add the Cloud SQL Instance User role to the IAM account logged into this notebook. (Client role previously granted)

In [ ]:
# add Cloud SQL Instance User role to current logged in IAM user
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.instanceUser"

Now the current IAM user can be added to the Cloud SQL instance as an IAM database user.

In [ ]:
# add current logged in IAM user to database
!gcloud sql users create {current_user[0]} \
  --instance={instance_name} \
  --type=cloud_iam_user

Finally, let's update our `getconn` function to connect to our Cloud SQL instance with IAM database authentication enabled.

> ⚠️ The below sample is a limited example as it logs in to the Cloud SQL instance and outputs the current time. By default new IAM database users have no permissions on a Cloud SQL instance. To connect to specific tables and perform more complex queries, permissions must be granted at the database level. ([Grant Database Privileges to the IAM user](https://cloud.google.com/sql/docs/mysql/add-manage-iam-users#grant-db-privileges))

In [ ]:
from google.cloud.sql.connector import Connector
import google.auth
from google.auth.transport.requests import Request
import sqlalchemy

# IAM database user parameter (IAM user's email before the "@" sign, mysql truncates usernames)
# ex. IAM user with email "demo-user@test.com" would have database username "demo-user"
IAM_USER = current_user[0].split("@")[0]

# initialize connector
connector = Connector()

# getconn now using IAM user and requiring no password with IAM Auth enabled
def getconn():
    conn = connector.connect(
      INSTANCE_CONNECTION_NAME,
      "pymysql",
      user=IAM_USER,
      db="", # log in to instance but don't connect to specific database
      enable_iam_auth=True
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# connect to connection pool
with pool.connect() as db_conn:
    # get current datetime from database
    results = db_conn.execute(sqlalchemy.text("SELECT NOW()")).fetchone()

    # output time
    print("Current time: ", results[0])

# cleanup connector
connector.close()

Sucess! You were able to connect to Cloud SQL as an IAM authenticated user using the Cloud SQL Python Connector! 🍾 👏 🏆

<img src="https://media.giphy.com/media/YTbZzCkRQCEJa/giphy.gif" />

## 🗑 Clean Up Notebook Resources
Make sure to delete your Cloud SQL instance when your are finished with this notebook to avoid further costs. 💸 💰

In [ ]:
# delete Cloud SQL instance
!gcloud sql instances delete {instance_name}

## ✍ **Appendix**
Additional information provided for connecting to a Cloud SQL instance using private IP connections.


### 🔒 **Using Private IP Connections**
By default the connector connects to the Cloud SQL instance database using a Public IP address.

**Private IP** connections are also supported by the connector and can be easily enabled through the `ip_type` parameter in the connector's `connect` method.

> ⚠️ To connect via Private IP, the Cloud SQL instance being connected to must have a Private IP address configured within a VPC Network. ([How to Configure Private IP](https://cloud.google.com/sql/docs/mysql/configure-private-ip))

> 🚫 The below cell is a working sample but will not work within this notebook as the notebook is not within your VPC Network! The cell should be copied into an environment (Cloud Run, Cloud Functions, App Engine etc.) that has access to the VPC Network.

 > [Connecting Cloud Run to a VPC Network](https://cloud.google.com/run/docs/configuring/connecting-vpc)

Let's update our `getconn` function to connect to our Cloud SQL instance with Private IP.

In [ ]:
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy

# initialize connector
connector = Connector()

# getconn now set to private IP
def getconn():
    conn = connector.connect(
      INSTANCE_CONNECTION_NAME, # <PROJECT-ID>:<REGION>:<INSTANCE-NAME>
      "pymysql",
      user=DB_USER,
      password=DB_PASS,
      db=DB_NAME,
      ip_type=IPTypes.PRIVATE
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# connect to connection pool
with pool.connect() as db_conn:
    # query database and fetch results
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

    # show results
    for row in results:
        print(row)

# cleanup connector
connector.close()